# **NLU First Assignment** 
*   **Zihadul Azam**
*   Id: 221747
*   zihadul.azam@studenti.unitn.it


### **Requirements**


*   SpaCy: run `pip install spacy`

In [17]:
example = "I saw the man with a telescope." 
example_2 = "I saw a man on a hill with a telescope."

import spacy

nlp = spacy.load('en')

### Visualize dependency graph with `displacy` (for test)

In [18]:
from spacy import displacy
doc = nlp(example)
displacy.render(doc, style="dep", jupyter=True, options={'distance': 90})

## **Function 1:** Extract a path of dependency relations from the ROOT to a token
##### **Input:** a sentence (type: string)


> Example: `"I saw the man with a telescope."`


##### **Output:** for each token the path will be a list of dependency relations, where first element is ROOT (type: dictionary)


> Example: `[{'token': 'I', 'path': ['ROOT(saw)', 'nsubj(I)']}, ... ]`




---

The goal of this function is to find all the paths from the ROOT to the tokens. So, if we have **N words** in the input sentence, the output of this function will be **N paths**.
However, I found two solutions:


1.   The first one is very handy, I used the **ancestors' list** of the token. SpaCy library provides us a list of ancestors for each token (`token.ancestors` property), but in reverse order (the first token in the list is the nearest ancestor). So, for having the path in top-bottom format (from ROOT to the token) I took each ancestor and pushed `token.dep_`(relation) and `token.text` into a list.
2.   The second solution is the classic one, I used the **head** parameter of the token. The function goes up recursively until not find the ROOT. How we can know that the current token is the ROOT token? If **`token.head` is equal to the current token**, then the current token is the ROOT token. However, we can find the ROOT also by using `token.sent.root`.



### **Solution 1:** using `token.ancestors`

In [19]:
def extract_all_paths_from_root(sentence):
  """
  this function extract dependency path
  from the root to each token
  """
  doc = nlp(sentence)
  paths = {}

  for sent in doc.sents:
    for token in sent:
      paths[token.i] = {'token': token.text, 'path': extract_path_from_root(token)} #for each token extract path
  return paths

def extract_path_from_root(token):
  """
  this function extract path from the root
  to a token using ancestor list of the token
  """
  path=["{}({})".format(token.dep_, token.text)] #add current token
  for ances_token in token.ancestors:
      path.insert(0, "{}({})".format(ances_token.dep_, ances_token.text)) #push ancestor token info 
  return path

In [20]:
res = extract_all_paths_from_root(example)
print('---------------------Al paths from ROOT to tokens---------------------')
print('Input sentence: ', example)
print('----------------------------------------------------------------------')
for key in res:
  print(res[key])

---------------------Al paths from ROOT to tokens---------------------
Input sentence:  I saw the man with a telescope.
----------------------------------------------------------------------
{'token': 'I', 'path': ['ROOT(saw)', 'nsubj(I)']}
{'token': 'saw', 'path': ['ROOT(saw)']}
{'token': 'the', 'path': ['ROOT(saw)', 'dobj(man)', 'det(the)']}
{'token': 'man', 'path': ['ROOT(saw)', 'dobj(man)']}
{'token': 'with', 'path': ['ROOT(saw)', 'dobj(man)', 'prep(with)']}
{'token': 'a', 'path': ['ROOT(saw)', 'dobj(man)', 'prep(with)', 'pobj(telescope)', 'det(a)']}
{'token': 'telescope', 'path': ['ROOT(saw)', 'dobj(man)', 'prep(with)', 'pobj(telescope)']}
{'token': '.', 'path': ['ROOT(saw)', 'punct(.)']}


### **Solution 2:** using `token.head` (recursive)

In [21]:
def extract_all_paths_from_root_rec(sentence):
  """
  this function extract dependency path
  from the root to each token using head of the token.
  """
  doc = nlp(sentence)
  paths = {}
  for sent in doc.sents:
    for token in sent:
      paths[token.i] = {'token': token.text, 'path': extract_path_from_root_rec(token)} #for each token extract path
  return paths

def extract_path_from_root_rec(token):
  """
  this function is a recursive function.
  recursively go up until find the root
  """
  result = ["{}({})".format(token.dep_, token.text)] #add current token
  # if this token is root token
  if token.head == token:
    return result
  return extract_path_from_root_rec(token.head) + result # joint ancestor token info with current token info and return

In [22]:
res = extract_all_paths_from_root_rec(example)
print('---------------------Al paths from ROOT to tokens (recursive)---------------------')
print('Input sentence: ', example)
print('----------------------------------------------------------------------------------')
for key in res:
  print(res[key])

---------------------Al paths from ROOT to tokens (recursive)---------------------
Input sentence:  I saw the man with a telescope.
----------------------------------------------------------------------------------
{'token': 'I', 'path': ['ROOT(saw)', 'nsubj(I)']}
{'token': 'saw', 'path': ['ROOT(saw)']}
{'token': 'the', 'path': ['ROOT(saw)', 'dobj(man)', 'det(the)']}
{'token': 'man', 'path': ['ROOT(saw)', 'dobj(man)']}
{'token': 'with', 'path': ['ROOT(saw)', 'dobj(man)', 'prep(with)']}
{'token': 'a', 'path': ['ROOT(saw)', 'dobj(man)', 'prep(with)', 'pobj(telescope)', 'det(a)']}
{'token': 'telescope', 'path': ['ROOT(saw)', 'dobj(man)', 'prep(with)', 'pobj(telescope)']}
{'token': '.', 'path': ['ROOT(saw)', 'punct(.)']}


## **Function 2:** Extract subtree of a dependents given a token
##### **Input:** a sentence


> Example: `"I saw the man with a telescope."`


##### **Output:** for each token in Doc objects you extract a subtree of its dependents as a list (ordered w.r.t. sentence order)


> Example: `[{'token': 'man', 'subtree': ['the', 'with', 'a', 'telescope']}, ...]`


---


The goal of this function is to find the subtree of its dependents (ordered w.r.t. sentence order). For having the subtree for each token I used `token.subtree` property, which returns: token + its dependents. But, reading carefully the task requirements I interpreted that, the output should contain only the list of its dependents, not the token itself, so I removed the root token (current token) from the subtree with an if condition: `if sub_token is not token`.

In [23]:
def get_subtree_of_a_token(token):
  return [sub_token.text for sub_token in token.subtree if sub_token is not token] #return subtree without root

def extract_subtree(sentence):
  doc = nlp(sentence)
  result = {}

  for sent in doc.sents:
    for token in sent:
      result[token.i] = {'token': token.text, 'subtree': get_subtree_of_a_token(token)}
  return result

In [24]:
res = extract_subtree(example)
print('---------------------Subtree for each tokens---------------------')
print('Input sentence: ', example,)
print('-----------------------------------------------------------------')
for key in res:
  print(res[key])

---------------------Subtree for each tokens---------------------
Input sentence:  I saw the man with a telescope.
-----------------------------------------------------------------
{'token': 'I', 'subtree': []}
{'token': 'saw', 'subtree': ['I', 'the', 'man', 'with', 'a', 'telescope', '.']}
{'token': 'the', 'subtree': []}
{'token': 'man', 'subtree': ['the', 'with', 'a', 'telescope']}
{'token': 'with', 'subtree': ['a', 'telescope']}
{'token': 'a', 'subtree': []}
{'token': 'telescope', 'subtree': ['a']}
{'token': '.', 'subtree': []}


## **Function 3:** Check if a given list of tokens (segment of a sentence) forms a subtree
##### **Input:** a sentence | ordered list of words from a sentence


> Example sentence: `"I saw the man with a telescope."`

> Example token list: `['a', 'telescope']`


##### **Output:** True/False based on the sequence forming a subtree or not


> Example: True


---



The goal of this function is, given a sentence and a segment of sentence (list of tokens), find if the segment forms a subtree or not. 

To find whether it forms or not, first, we have to take the input sentence and find all dependency subtrees (for each token). I could have used **function number 2** to get all subtrees, but It does not include the root token (due to specific requirement) when returning a subtree. So, here I implemented a new function that includes also the root token when return a subtree (used `token.subtree` property). 

Once, we have all the subtrees we can compare them with our input token list (sentence segment). To compare two lists I used `if list1 == list2`.

In [25]:
def is_subtree(sentence, tokens=[]):
  trees = extract_subtree_with_root(sentence)
  for key in trees:
    if trees[key]['subtree'] == tokens:
      return True
  return False

def extract_subtree_with_root(sentence):
  doc = nlp(sentence)
  result = {}

  for sent in doc.sents:
    for token in sent:
      sub_tree = [sub_token.text for sub_token in token.subtree]
      result[token.i] = {'token': token.text, 'subtree': sub_tree}
  return result

In [26]:
print('---------------------Check if a given list of tokens forms a subtree---------------------')
print('Input sentence: ', example,)
print('-----------------------------------------------------------------------------------------')
seg_sent = ['a', 'telescope']
seg_sent_str = ' '.join(seg_sent)
print('Test 1 with segment of token: ', seg_sent)
print('Is <', seg_sent_str, '> form a subtree?: ', is_subtree(example, seg_sent))
print('-----------------------------------------------------------------------------------------')
seg_sent = ['telescope', 'a']
seg_sent_str = ' '.join(seg_sent)
print('Test 2 with segment of token: ', seg_sent)
print('Is <', seg_sent_str, '> form a subtree?: ', is_subtree(example, seg_sent))
print('-----------------------------------------------------------------------------------------')
seg_sent = ['I', 'the', 'saw', 'man']
seg_sent_str = ' '.join(seg_sent)
print('Test 2 with segment of token: ', seg_sent)
print('Is <', seg_sent_str, '> form a subtree?: ', is_subtree(example, seg_sent))

---------------------Check if a given list of tokens forms a subtree---------------------
Input sentence:  I saw the man with a telescope.
-----------------------------------------------------------------------------------------
Test 1 with segment of token:  ['a', 'telescope']
Is < a telescope > form a subtree?:  True
-----------------------------------------------------------------------------------------
Test 2 with segment of token:  ['telescope', 'a']
Is < telescope a > form a subtree?:  False
-----------------------------------------------------------------------------------------
Test 2 with segment of token:  ['I', 'the', 'saw', 'man']
Is < I the saw man > form a subtree?:  False


## **Function 4:** Identify head of a span, given its tokens 
##### **Input:** Span - is a sequence of words (not necessarily a sentence)


> Example: `'a man with a telescope'`


##### **Output:** the head of the span (single word)


> Example: `'man'`


---

This function returns the **head of a span**. To find the head token we have to find the token which links itself as the head. So we iterate over all tokens and return token `if token.head == token`.


In [27]:
def get_head_of_span(span):
  doc = nlp(span)
  for sent in doc.sents:
    for token in sent:
      if token.head == token:
        return token.text
  return None

In [28]:
print('---------------------Identify head of a span---------------------')
span = 'a man with a telescope'
print('Head of span <', span, '> is: ', get_head_of_span(span))

---------------------Identify head of a span---------------------
Head of span < a man with a telescope > is:  man


## **Function 5:** Extract sentence subject, direct object and indirect object spans 
##### **Input:** a sequence


> Example: `"I saw the man with a telescope."`


##### **Output:**  lists of words that form a span (not a single word) for subject, direct object, and indirect object (if present of course, otherwise empty) - Type: dict of lists


> Example: `nsubj :	 ['I']` `dobj :	 ['man']` `dative :	 []`


---

This functions returns a dictionary with three lists:


1.   **Subject token list**: includes all the subject tokens of the input sentence. A token is a subject `if token.dep_ == 'nsubj'`
2.   **Direct object list**: includes all the direct object tokens of the input sentence. A token is a direct object `if token.dep_ == 'dobj'`
3.   **Indirect object list**: include all the indirect object tokens of the input sentence. A token is a indirect object `if token.dep_ == 'dative'`




In [29]:
def extract_S_DO_IO(sentence):
  doc = nlp(sentence)

  subject_tag = 'nsubj'
  direct_obj_tag = 'dobj'
  indirect_obj_tag = 'dative'
  filter_tags = [subject_tag, direct_obj_tag, indirect_obj_tag]

  result = {
      subject_tag: [],
      direct_obj_tag: [],
      indirect_obj_tag: [],
  }

  for sent in doc.sents:
    for token in sent:
      if token.dep_ in filter_tags:
        result[token.dep_].append(token.text)
  return result

In [30]:
print('---------------------Extract sentence subject, direct object and indirect object spans---------------------')
print('Test 1')
res = extract_S_DO_IO(example)
print('Subject, direct object, and indirect object of the sentence: <', example,'>')
for key in res:
  print(key, ':\t', res[key])
print('-----------------------------------------------------------------------------------------')
example = 'Give the book to me.'
print('Test 2')
res = extract_S_DO_IO(example)
print('Subject, direct object, and indirect object of the sentence: <', example,'>')
for key in res:
  print(key, ':\t', res[key])

---------------------Extract sentence subject, direct object and indirect object spans---------------------
Test 1
Subject, direct object, and indirect object of the sentence: < I saw the man with a telescope. >
nsubj :	 ['I']
dobj :	 ['man']
dative :	 []
-----------------------------------------------------------------------------------------
Test 2
Subject, direct object, and indirect object of the sentence: < Give the book to me. >
nsubj :	 []
dobj :	 ['book']
dative :	 ['to']
